# Homework 1: Preprocessing and Text Classification

Student Name: Nodens Koren

Student ID: 1060811

# General Info

<b>Due date</b>: Sunday, 5 Apr 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection tweets. The task is to classify whether a tweet constitutes a rumour event. This homework involves writing code to preprocess data and perform text classification.

# 1. Preprocessing (5 marks)

**Instructions**: Run the code below to download the tweet corpus for the assignment. Note: the download may take some time. **No implementation is needed.**

In [1]:
import requests
import os
from pathlib import Path
!pip install nltk

fname = 'rumour-data.tgz'
data_dir = os.path.splitext(fname)[0] #'rumour-data'

my_file = Path(fname)
if not my_file.is_file():
    url = "https://github.com/jhlau/jhlau.github.io/blob/master/files/rumour-data.tgz?raw=true"
    r = requests.get(url)

    #Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)
        
print("Done. File downloaded:", my_file)


Done. File downloaded: rumour-data.tgz


**Instructions**: Run the code to extract the zip file. Note: the extraction may take a minute or two. **No implementation is needed.**

In [2]:
import tarfile

#decompress rumour-data.tgz
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

#remove superfluous files (e.g. .DS_store)
extra_files = []
for r, d, f in os.walk(data_dir):
    for file in f:
        if (file.startswith(".")):
            extra_files.append(os.path.join(r, file))
for f in extra_files:
    os.remove(f)

print("Extraction done.")

Extraction done.


### Question 1 (1.0 mark)

**Instructions**: The corpus data is in the *rumour-data* folder. It contains 2 sub-folders: *non-rumours* and *rumours*. As the names suggest, *rumours* contains all rumour-propagating tweets, while *non-rumours* has normal tweets. Within  *rumours* and *non-rumours*, you'll find some sub-folders, each named with an ID. Each of these IDs constitutes an 'event', where an event is defined as consisting a **source tweet** and its **reactions**.

An illustration of the folder structure is given below:

    rumour-data
        - rumours
            - 498254340310966273
                - reactions
                    - 498254340310966273.json
                    - 498260814487642112.json
                - source-tweet
                    - 498254340310966273.json
        - non-rumours

Now we need to gather the tweet messages for rumours and non-rumour events. As the individual tweets are stored in json format, we need to use a json parser to parse and collect the actual tweet message. The function `get_tweet_text_from_json(file_path)` is provided to do that.

**Task**: Complete the `get_events(event_dir)` function. The function should return **a list of events** for a particular class of tweets (e.g. rumours), and each event should contain the source tweet message and all reaction tweet messages.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [3]:
import json

def get_tweet_text_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
        return data["text"]
    
def get_events(event_dir):
    event_list = []
    for event in sorted(os.listdir(event_dir)):
        ###
        # Your answer BEGINS HERE
        ###
        message_list = []
        source_dir = os.path.join(event_dir, event, "source-tweet")
        reaction_dir = os.path.join(event_dir, event, "reactions")
        
        # extract the source tweet
        for source_tweet in os.listdir(source_dir):
            message_list.append(get_tweet_text_from_json(os.path.join(source_dir, source_tweet)))
        
        # extract the reaction tweets
        for reaction_tweet in os.listdir(reaction_dir):
            message_list.append(get_tweet_text_from_json(os.path.join(reaction_dir, reaction_tweet)))
        
        # append the single preprocessed event to the event list
        event_list.append(message_list)
        
        ###
        # Your answer ENDS HERE
        ###
    return event_list

#a list of events, and each event is a list of tweets (source tweet + reactions)    
rumour_events = get_events(os.path.join(data_dir, "rumours"))
nonrumour_events = get_events(os.path.join(data_dir, "non-rumours"))

print("Number of rumour events =", len(rumour_events))
print("Number of non-rumour events =", len(nonrumour_events))

Number of rumour events = 500
Number of non-rumour events = 1000


**For your testing:**

In [4]:
assert(len(rumour_events) == 500)
assert(len(nonrumour_events) == 1000)

### Question 2 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation. The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); and (2) remove stopwords (based on NLTK `stopwords`).

**Task**: Complete the `preprocess_events(event)` function. The function takes **a list of events** as input, and returns **a list of preprocessed events**. Each preprocessed event should have a dictionary of words and frequencies.

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [5]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter

tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))

def preprocess_events(events):
    ###
    # Your answer BEGINS HERE
    ###
    preprocessed_event_list = []

    # Tokenize each event and remove the stopwords in each event
    for event in events:
        filtered_sentence = [w for w in tt.tokenize(' '.join(event)) if not w in stopwords]
        frequency = dict(Counter(filtered_sentence))
        preprocessed_event_list.append(frequency)    
    
    return preprocessed_event_list
    ###
    # Your answer ENDS HERE
    ###

preprocessed_rumour_events = preprocess_events(rumour_events)
preprocessed_nonrumour_events = preprocess_events(nonrumour_events)

print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noden\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Number of preprocessed rumour events = 500
Number of preprocessed non-rumour events = 1000


**For your testing**:

In [6]:
assert(len(preprocessed_rumour_events) == 500)
assert(len(preprocessed_nonrumour_events) == 1000)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [7]:
def get_all_hashtags(events):
    hashtags = set([])
    for event in events:
        for word, frequency in event.items():
            if word.startswith("#"):
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(preprocessed_rumour_events + preprocessed_nonrumour_events)
print("Number of hashtags =", len(hashtags))

Number of hashtags = 1829


### Question 3 (2.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing a reversed version of the MaxMatch algorithm discussed in class, where matching begins at the end of the hashtag and progresses backwards. NLTK has a list of words that you can use for matching, see starter code below. Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching. When lemmatising a word, you also need to provide the part-of-speech tag of the word. You should use `nltk.tag.pos_tag` for doing part-of-speech tagging.

Note that the list of words is incomplete, and, if you are unable to make any longer match, your code should default to matching a single letter. Create a new list of tokenized hashtags (this should be a list of lists of strings) and use slicing to print out the last 20 hashtags in the list.

For example, given "#speakup", the algorithm should produce: \["#", "speak", "up"\]. And note that you do not need to delete the hashtag symbol ("#") from the tokenised outputs.

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing a reversed MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of word tokens".

**Check**: Use the assertion statements in *"For your testing"* below for the expected output.

In [31]:
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
nltk.download('words')
nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
case_insensitive_words = set(map(lambda x:x.lower(), words)) # a list of "caseless" words mapped from wordnet

# This function maps the NLTK pos-tag format to the Wordnet pos-tag format
# This function is borrowed from https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
# end of the borrowed section

def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    tokenized_hashtags = {}
    for hashtag in hashtags:
        tail = len(hashtag)
        hashtag_token = []
        while tail > 0:
            word = None
            
            # Starts matching from the longest substring
            for i in range(len(hashtag), 0, -1):
                head = tail - i
                if head < 0:
                    continue
                else:
                    substring = hashtag[head:tail]
                    
                    # Perform a case-insensitive string check between the current substring and the words in wordnet
                    # If the substring matches a word in wordnet, append it to the token list, and start checking the remaining
                    # substring from a new head
                    if lemmatizer.lemmatize(substring.lower(), get_wordnet_pos(substring.lower())) in case_insensitive_words:
                        word = substring
                        hashtag_token.append(word)
                        tail -= i
                        break
                        
            # If no substring matches a word at all, tokenize the last character
            if word is None:
                hashtag_token.append(hashtag[tail-1])
                tail -= 1
        
        tokenized_hashtags[hashtag] = hashtag_token[::-1]

    return tokenized_hashtags    
    ###
    # Your answer ENDS HERE
    ###  

tokenized_hashtags = tokenize_hashtags(hashtags)

print(list(tokenized_hashtags.items())[:20])

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\noden\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\noden\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\noden\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('#MaryamrajaviI', ['#', 'Mar', 'yam', 'raja', 'v', 'i', 'I']), ('#wolvesinsheepsclothing', ['#', 'wolve', 'sin', 'sheeps', 'clothing']), ('#roft', ['#', 'r', 'oft']), ('#CoPilot', ['#', 'CoPilot']), ('#FergusonLies', ['#', 'Ferguson', 'Lies']), ('#developing', ['#', 'developing']), ('#ONUS', ['#', 'ONUS']), ('#smh', ['#', 's', 'm', 'h']), ('#FilthyCanadaUnderSiege', ['#', 'Filthy', 'Canada', 'Under', 'Siege']), ('#doughnutchase', ['#', 'doughnut', 'chase']), ('#kmov', ['#', 'k', 'mo', 'v']), ('#radicalism', ['#', 'radicalism']), ('#ninjaplease', ['#', 'ninja', 'please']), ('#NorthKorea', ['#', 'North', 'Ko', 'rea']), ('#PrayForSydneyHostages', ['#', 'Pray', 'For', 'Syd', 'n', 'ey', 'Hostages']), ('#newsspeak', ['#', 'news', 'speak']), ('#MilitantIslamists', ['#', 'Militant', 'Islamists']), ('#dicks', ['#', 'dicks']), ('#alive', ['#', 'alive']), ('#bcot', ['#', 'b', 'cot'])]


**For your testing:**

In [32]:
assert(len(tokenized_hashtags) == len(hashtags))

### Question 4 (1.0 mark)

**Instructions**: Now that we have the tokenized hashtags, we need to go back and update the bag-of-words representation for each event.

**Task**: Complete the ``update_event_bow(events)`` function. The function takes **a list of preprocessed events**, and for each event, it looks for every hashtag it has and updates the bag-of-words dictionary with the tokenized hashtag tokens. Note: you do not need to delete the counts of the original hashtags when updating the bag-of-words (e.g., if a document has "#speakup":2 in its bag-of-words representation, you do not need to delete this hashtag and its counts).

In [33]:
def update_event_bow(events):
    ###
    # Your answer BEGINS HERE
    ###
    updated_event_list = []
    for event in events:
        for hashtag in get_all_hashtags([event]):
            
            # Update the token count with additional tokenized hashtags.
            # If the token exists already, simply add one to the count. Otherwise create a new dictionary entry.
            for token in tokenized_hashtags[hashtag]:
                if token in event:
                    event[token] += 1
                else:
                    event[token] = 1
    ###
    # Your answer ENDS HERE
    ###
    
update_event_bow(preprocessed_rumour_events)
update_event_bow(preprocessed_nonrumour_events)

print("Number of preprocessed rumour events =", len(preprocessed_rumour_events))
print("Number of preprocessed non-rumour events =", len(preprocessed_nonrumour_events))

Number of preprocessed rumour events = 500
Number of preprocessed non-rumour events = 1000


# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict, given a tweet and its reactions, whether it is a rumour or not. The task here is to create training, development and test partitions from the preprocessed events and convert the bag-of-words representation into feature vectors.

**Task**: Using scikit-learn, create training, development and test partitions with a 60%/20%/20% ratio. Remember to preserve the ratio of rumour/non-rumour events for all your partitions. Next, turn the bag-of-words dictionary of each event into a feature vector, using scikit-learn `DictVectorizer`.

In [44]:
from sklearn.feature_extraction import DictVectorizer
import random
import numpy as np
vectorizer = DictVectorizer()

###
# Your answer BEGINS HERE
###

# Randomly shuffle the rumor and non-rumor events to make the training process less biased
random.shuffle(preprocessed_rumour_events)
random.shuffle(preprocessed_nonrumour_events)

# Partition a training set with 60% events from the both the rumor events and the non-rumor events
# Create a label set for the training set
training_set = preprocessed_rumour_events[:int(len(preprocessed_rumour_events)*.6)] + preprocessed_nonrumour_events[:int(len(preprocessed_nonrumour_events)*.6)]
training_label = ['rumour'] * int(len(preprocessed_rumour_events)*.6) + ['non-rumour'] * int(len(preprocessed_nonrumour_events)*.6)

# Partition a training set with 20% events from the remaining 40% rumor and non-rumor events
# Create a label set for the development set
development_set = preprocessed_rumour_events[int(len(preprocessed_rumour_events)*.6):int(len(preprocessed_rumour_events)*.8)] + preprocessed_nonrumour_events[int(len(preprocessed_nonrumour_events)*.6):int(len(preprocessed_nonrumour_events)*.8)]
development_label = ['rumour'] * int(len(preprocessed_rumour_events)*.2) + ['non-rumour'] * int(len(preprocessed_nonrumour_events)*.2)

# Create a test set with the remaining 20% rumor and non-rumor events
# Create a label set for the test set
test_set = preprocessed_rumour_events[int(len(preprocessed_rumour_events)*.8):int(len(preprocessed_rumour_events))] + preprocessed_nonrumour_events[int(len(preprocessed_nonrumour_events)*.8):int(len(preprocessed_nonrumour_events))]
test_label = ['rumour'] * int(len(preprocessed_rumour_events)*.2) + ['non-rumour'] * int(len(preprocessed_nonrumour_events)*.2)

training_data = vectorizer.fit_transform(training_set)
development_data = vectorizer.transform(development_set)
test_data = vectorizer.transform(test_set)
###
# Your answer ENDS HERE
###

print("Vocabulary size =", len(vectorizer.vocabulary_))

Vocabulary size = 27684


### Question 6 (2.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation (hyper)parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the accuracy with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance for different hyper-parameter settings.

In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy

###
# Your answer BEGINS HERE
###

# Performs a grid search on the "alpha" hyperparamater of the Naive Bayes model
# Accuracy is tested with the development set
best_alpha = 0
best_accuracy = 0
print('>>>>>>>>>>>>>>>>>>>>>>>> Naive Bayes <<<<<<<<<<<<<<<<<<<<<<<<')
for a in numpy.arange(0, 5, 0.01):
    naive_bayes_model = MultinomialNB(alpha=a, fit_prior=True, class_prior=None)
    naive_bayes_model.fit(training_data, np.array(training_label))
    if accuracy_score(naive_bayes_model.predict(development_data), np.array(development_label)) > best_accuracy:
        best_accuracy = accuracy_score(naive_bayes_model.predict(development_data), np.array(development_label))
        best_alpha = a
        print('Best Accuracy: ', best_accuracy, ' alpha = ', best_alpha)

# Train a final Naive Bayes model with the alpha value that gives the best result
naive_bayes_model = MultinomialNB(alpha=best_alpha, fit_prior=True, class_prior=None).fit(training_data, np.array(training_label))

# Performs a grid search on the "C" hyperparamater of the Logistic Regression model
# Accuracy is tested with the development set
best_c = 0
best_accuracy_2 = 0
print('\n>>>>>>>>>>>>>>>>>>>> Logistic Regression <<<<<<<<<<<<<<<<<<<<')
for c in numpy.arange(0.01, 5, 0.01):
    logistic_regression_model = LogisticRegression(C=c)
    logistic_regression_model.fit(training_data, np.array(training_label))
    if accuracy_score(logistic_regression_model.predict(development_data), np.array(development_label)) > best_accuracy_2:
        best_accuracy_2 = accuracy_score(logistic_regression_model.predict(development_data), np.array(development_label))
        best_c = c
        print('Best Accuracy: ', best_accuracy_2, ' C = ', best_c)


logistic_regression_model = LogisticRegression(C=c).fit(training_data, np.array(training_label))
###
# Your answer ENDS HERE
###

>>>>>>>>>>>>>>>>>>>>>>>> Naive Bayes <<<<<<<<<<<<<<<<<<<<<<<<
Best Accuracy:  0.74  alpha =  0.0
Best Accuracy:  0.7666666666666667  alpha =  0.01
Best Accuracy:  0.7766666666666666  alpha =  0.02
Best Accuracy:  0.78  alpha =  0.03
Best Accuracy:  0.79  alpha =  0.04
Best Accuracy:  0.7933333333333333  alpha =  0.05
Best Accuracy:  0.7966666666666666  alpha =  0.2
Best Accuracy:  0.8  alpha =  0.23
Best Accuracy:  0.8033333333333333  alpha =  0.36
Best Accuracy:  0.81  alpha =  0.37


C:\Users\noden\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)



>>>>>>>>>>>>>>>>>>>> Logistic Regression <<<<<<<<<<<<<<<<<<<<
Best Accuracy:  0.8  C =  0.01
Best Accuracy:  0.8166666666666667  C =  0.02
Best Accuracy:  0.8266666666666667  C =  0.03


C:\Users\noden\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best Accuracy:  0.83  C =  0.060000000000000005
Best Accuracy:  0.8333333333333334  C =  0.06999999999999999
Best Accuracy:  0.84  C =  0.09999999999999999
Best Accuracy:  0.8433333333333334  C =  0.13
Best Accuracy:  0.85  C =  0.3
Best Accuracy:  0.8533333333333334  C =  4.37


### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both accuracy and macro-averaged F-score for each classifier. Be sure to label your output.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using optimal hyper-parameter settings.

In [46]:
from sklearn.metrics import f1_score

###
# Your answer BEGINS HERE
###

print('>>>>>>>>>>>>>>>>>>>>>>>> Naive Bayes <<<<<<<<<<<<<<<<<<<<<<<<')
print("Accuracy: ", accuracy_score(naive_bayes_model.predict(test_data), np.array(test_label)))
print("Macro-averaged F-score: ", f1_score(naive_bayes_model.predict(test_data), np.array(test_label), average='macro'))

print('\n>>>>>>>>>>>>>>>>>>>> Logistic Regression <<<<<<<<<<<<<<<<<<<<')
print("Accuracy: ", accuracy_score(logistic_regression_model.predict(test_data), np.array(test_label)))
print("Macro-averaged F-score: ", f1_score(logistic_regression_model.predict(test_data), np.array(test_label), average='macro'))

###
# Your answer ENDS HERE
###

>>>>>>>>>>>>>>>>>>>>>>>> Naive Bayes <<<<<<<<<<<<<<<<<<<<<<<<
Accuracy:  0.81
Macro-averaged F-score:  0.7925537722458783

>>>>>>>>>>>>>>>>>>>> Logistic Regression <<<<<<<<<<<<<<<<<<<<
Accuracy:  0.8433333333333334
Macro-averaged F-score:  0.8223980048116285
